In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
from matplotlib import pyplot as plt
from scipy import stats
from sklearn import preprocessing, svm
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import statsmodels.api as sm
from statsmodels.iolib.table import SimpleTable
from sklearn.metrics import r2_score

In [33]:
cd mlmodel/

/home/jupyter/mlmodel


In [36]:
train_data = pd.read_csv('test1.csv', delimiter='\t',parse_dates=['DateObserve'])

In [ ]:
train_data['DateObserve']=pd.to_datetime(train_data['DateObserve']).apply(lambda x: x.strftime('%Y-%m-%d'))

In [ ]:
train_data.head()

In [ ]:
count_nan=train_data.groupby(['WebPriceId','StockStatus']).count().reset_index()

##### Фильтруем id по количеству записей (более 150)

In [ ]:
count_filtered = train_data.groupby(['WebPriceId'])['WebPriceId'].count()

In [ ]:
count_filtered.head()

In [ ]:
filtered_ids = pd.DataFrame(count_filtered[count_filtered > 110].index)

In [ ]:
filtered_ids.head()

In [ ]:
train_data_1 = filtered_ids.merge(train_data, on='WebPriceId',how='inner')
train_data_1.head(15)

In [ ]:
print(f'Отфильтрованное количество записей = {len(train_data_1)}')

In [ ]:
train_data_1.fillna(method='ffill', inplace=True)

In [ ]:
train_data_1=train_data_1.drop_duplicates()

In [ ]:
train_data_1.count()

In [ ]:
train_data_1=train_data_1[['WebPriceId','DateObserve','CurrentPrice']]


In [ ]:
train_data_1.head()

#### Делаем ряд стационарным

### Рассчет ипц

In [ ]:
get_mode = lambda x: x.value_counts(dropna=False).index[0]

In [ ]:
train_data_2=train_data_1.groupby(['WebPriceId','DateObserve']).agg({'CurrentPrice': ['mean']}).reset_index()

In [ ]:
train_data_2['DateObserve']=pd.to_datetime(train_data_2['DateObserve']).apply(lambda x: x.strftime('%Y-%m'))

In [ ]:
train_data_2.head()

In [ ]:
train_data_2.columns=['WebPriceId','DateObserve','CurrentPrice']

In [ ]:
train_data_2=train_data_2.groupby(['WebPriceId','DateObserve']).agg({'CurrentPrice': ['mean']}).reset_index()

In [ ]:
train_data_2.columns=['Id','Date','Price']

In [ ]:
train_data_2.head(10000)

In [ ]:
train_data_2['Price']=train_data_2['Price'].astype(int)

In [ ]:
itog = train_data_2.describe()
train_data_2.hist()
itog

In [ ]:
uni_data = train_data_2['Price']
uni_data.index = train_data_2['Date']
row =  [u'JB', u'p-value', u'skew', u'kurtosis']
jb_test = sm.stats.stattools.jarque_bera(uni_data)
a = np.vstack([jb_test])
itog = SimpleTable(a, row)
print(itog)

In [ ]:
test = sm.tsa.adfuller(uni_data)
print('adf: ', test[0]) 
print('p-value: ', test[1])
print('Critical values: ', test[4])
if test[0]> test[4]['5%']: 
    print('есть единичные корни, ряд не стационарен')
else:
    print('единичных корней нет, ряд стационарен')

In [ ]:
train_data_2.to_csv('train_data.csv',sep=';',encoding='utf-8')

In [ ]:
uni_data = np.log(train_data_2['Price'])
uni_data.index = train_data_2['Date']
uni_data.plot(subplots=True);